In [1]:
import os
import time
import glob
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
import matplotlib.pyplot as plt
from skimage import measure
from skimage.io import imread, imsave, imshow
from skimage.transform import resize
from skimage.filters import gaussian
from skimage.morphology import dilation, disk
from skimage.draw import polygon, polygon_perimeter
import cv2
import keras_cv

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Using TensorFlow backend


In [2]:
import tqdm
import random
import pathlib
import itertools
import collections

import os
import cv2
import numpy as np
import remotezip as rz

import tensorflow as tf

# Some modules to display an animation using imageio.
import imageio
from IPython import display
from urllib import request
from tensorflow_docs.vis import embed


In [3]:
cap = cv2.VideoCapture('/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Есть телефон/00_51_01.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Выполните обработку каждого кадра с использованием загруженной модели
    # Например, используйте model.predict(frame) для определения объектов на кадре

    # Отображение кадра
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [4]:
# Функция для получения списка файлов в папке
def list_files_in_folder(folder_path):
    file_list = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_list.append(os.path.join(root, file))
    return file_list

# Получите списки файлов из обеих папок
files_in_folder1 = list_files_in_folder('/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Есть телефон/')
files_in_folder2 = list_files_in_folder('/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Нет телефона/')

# Объедините списки файлов
all_files = files_in_folder1 + files_in_folder2

# Выведите список всех файлов
for file in all_files:
    print(file)

/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Есть телефон/01_08_31.mp4
/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Есть телефон/20_40_48.mp4
/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Есть телефон/10_41_06.mp4
/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Есть телефон/22_11_46.mp4
/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Есть телефон/04_21_56.mp4
/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Есть телефон/20_01_01.mp4
/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Есть телефон/14_24_34.mp4
/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Есть телефон/21_14_41.mp4
/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Есть телефон/00_26_30.mp4
/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Есть телефон/10_11_19.mp4
/Users/kak

In [5]:
def get_class(fname):
  """ Retrieve the name of the class given a filename.

    Args:
      fname: Name of the file in the UCF101 dataset.

    Returns:
      Class that the file belongs to.
  """
  return fname.split('_')[-3]

In [6]:
def get_files_per_class(files):
  """ Retrieve the files that belong to each class.

    Args:
      files: List of files in the dataset.

    Returns:
      Dictionary of class names (key) and files (values). 
  """
  files_for_class = collections.defaultdict(list)
  for fname in files:
    class_name = get_class(fname)
    files_for_class[class_name].append(fname)
  return files_for_class

In [7]:
NUM_CLASSES = 4
FILES_PER_CLASS = 144

In [8]:
files_for_class = get_files_per_class(file)
classes = list(files_for_class.keys())

IndexError: list index out of range

In [ ]:
print('Num classes:', len(classes))
print('Num videos for class[0]:', len(files_for_class[classes[0]]))

NameError: name 'classes' is not defined

In [ ]:
def select_subset_of_classes(files_for_class, classes, files_per_class):
  """ Create a dictionary with the class name and a subset of the files in that class.

    Args:
      files_for_class: Dictionary of class names (key) and files (values).
      classes: List of classes.
      files_per_class: Number of files per class of interest.

    Returns:
      Dictionary with class as key and list of specified number of video files in that class.
  """
  files_subset = dict()

  for class_name in classes:
    class_files = files_for_class[class_name]
    files_subset[class_name] = class_files[:files_per_class]

  return files_subset

In [18]:

CONFIDENCE = 0.5
SCORE_THRESHOLD = 0.5
IOU_THRESHOLD = 0.5

# конфигурация нейронной сети
config_path = "cfg/yolov3.cfg"
# файл весов сети YOLO
weights_path = "weights/yolov3.weights"
# weights_path = "weights/yolov3-tiny.weights"

# загрузка всех меток классов (объектов)
labels = open("/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Есть телефон/00_26_30.mp4").read().strip().split("\n")
# генерируем цвета для каждого объекта и последующего построения
colors = np.random.randint(0, 255, size=(len(labels), 3), dtype="uint8")


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x81 in position 43: invalid start byte

In [ ]:
['phone',
 'Ipad',
 'Cup',
 ]

['phone', 'Ipad', 'Cup']

In [ ]:
def format_frames(frame, output_size):
  """
    Pad and resize an image from a video.

    Args:
      frame: Image that needs to resized and padded. 
      output_size: Pixel size of the output frame image.

    Return:
      Formatted frame with padding of specified output size.
  """
  frame = tf.image.convert_image_dtype(frame, tf.float32)
  frame = tf.image.resize_with_pad(frame, *output_size)
  return frame

In [27]:
import sys

def progress_bar(count, total, status=""):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = "=" * filled_len + "-" * (bar_len - filled_len)

    sys.stdout.write("[{}] {}%  {}\r".format(bar, percents, status))
    sys.stdout.flush()


def download_progress(block_num, block_size, total_size):
    progress_bar(block_num * block_size, total_size)
    

urlretrieve(url, output_file, reporthook=download_progress)
import os
import subprocess
import tarfile
from urllib.request import urlretrieve

url = "http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar"
output_directory = "./data"
output_file = os.path.join(output_directory, "voc2012_raw.tar")
extract_directory = os.path.join(output_directory, "voc2012_raw")

# Создание папок, если они не существуют
os.makedirs(output_directory, exist_ok=True)
os.makedirs(extract_directory, exist_ok=True)

# Загрузка архива
urlretrieve(url, output_file)

# Извлечение содержимого архива
with tarfile.open(output_file, "r") as tar:
    tar.extractall(path=extract_directory)

# Вывод содержимого извлеченной папки
for item in os.listdir(os.path.join(extract_directory, "VOCdevkit/VOC2012")):
    print(item)

ImageSets====================================================] 100.0%  
SegmentationClass
SegmentationObject
Annotations
JPEGImages


In [33]:
data_root = os.path.join(root, "pew_research_data")


In [34]:
import csv

def load_file(file_path):
    data = []
    with open(file_path, encoding="utf-8") as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data.append(row)
    return data

In [35]:
file_path = os.path.join(data_root, "/Users/kakotichi/Documents/GitHub/sf_data_science/Vacation/submission.csv")
file_data = load_file(file_path)

In [ ]:
seg_dataset = VOCSegmentation(seg_root, year="2012", image_set=image_set, transforms=transformations, download=True)


In [40]:

det_dataset = VOCDetection(det_root, year="2012", image_set=image_set, transforms=transformations, download=True)


100%|██████████| 1999639040/1999639040 [22:14<00:00, 1498903.70it/s] 


Extracting ./data/voc2012_det/VOCtrainval_11-May-2012.tar to ./data/voc2012_det


In [41]:
import torch
from torch.utils.data import Dataset
from torchvision.datasets import VOCSegmentation, VOCDetection

class CombinedVOC(Dataset):
    def __init__(self, seg_root, det_root, year, image_set, transforms=None, download=True):
        self.seg_dataset = VOCSegmentation(seg_root, year=year, image_set=image_set, transforms=transforms, download=download)
        self.det_dataset = VOCDetection(det_root, year=year, image_set=image_set, transforms=transforms, download=download)
        self.transforms = transforms

    def __len__(self):
        return min(len(self.seg_dataset), len(self.det_dataset))

    def __getitem__(self, idx):
        seg_data, seg_target = self.seg_dataset[idx]
        det_data, det_target = self.det_dataset[idx]

        if self.transforms is not None:
            seg_data, seg_target = self.transforms(seg_data, seg_target)
            det_data, det_target = self.transforms(det_data, det_target)

        return (seg_data, seg_target), (det_data, det_target)

seg_root = "path/to/seg_data"
det_root = "path/to/det_data"
year = "2012"
image_set = "trainval"
transformations = None
download = True

combined_dataset = CombinedVOC(seg_root, det_root, year, image_set, transforms=transformations, download=download)


100%|██████████| 1999639040/1999639040 [23:49<00:00, 1399067.84it/s] 


Extracting path/to/seg_data/VOCtrainval_11-May-2012.tar to path/to/seg_data


100%|██████████| 1999639040/1999639040 [25:33<00:00, 1303672.42it/s] 


Extracting path/to/det_data/VOCtrainval_11-May-2012.tar to path/to/det_data


In [50]:
import cv2
import numpy as np
import urllib.request

# Загрузка модели и весов YOLOv3
yolo_config = "/Users/kakotichi/Documents/GitHub/sf_data_science/Vacation/yolov3-tiny.cfg"
yolo_weights = "/Users/kakotichi/Documents/GitHub/sf_data_science/Vacation/yolov3.weights"
net = cv2.dnn.readNet(yolo_weights, yolo_config)

# Загрузка списка классов
class_file = "/Users/kakotichi/Documents/GitHub/sf_data_science/Vacation/coco.txt"
with open(class_file, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Получение выходных слоев для сети YOLO
layer_names = net.getLayerNames()
output_layers_indices = net.getUnconnectedOutLayers()
output_layers_indices = np.reshape(output_layers_indices, (output_layers_indices.size,))
out_layers = [layer_names[i - 1] for i in output_layers_indices]

# Устанавливаем классы, которые хотим обнаружить
classes_to_look_for = ["cell phone", "laptop", "tv", "remote"]  

In [7]:
import cv2
import numpy as np
import time

def detect_objects(frame, net, out_layers, classes_to_look_for):
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, scalefactor=1/255, size=(416, 416), mean=(0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    layer_outputs = net.forward(out_layers)

    class_ids = []
    confidences = []
    boxes = []

    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.4 and classes[class_id] in classes_to_look_for:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
                class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.4)

    detected_objects = []
    for i in indices:
        i = i[0]
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        detected_objects.append(label)
        confidence = round(confidences[i], 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, f'{label} {confidence}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
    return frame, detected_objects

yolo_config = "/Users/kakotichi/Documents/GitHub/sf_data_science/Vacation/yolov3-tiny.cfg"
yolo_weights = "/Users/kakotichi/Documents/GitHub/sf_data_science/Vacation/yolov3.weights"
net = cv2.dnn.readNet(yolo_weights, yolo_config)

class_file = "/Users/kakotichi/Documents/GitHub/sf_data_science/Vacation/coco.txt"
with open(class_file, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers_indices = net.getUnconnectedOutLayers()
output_layers_indices = np.reshape(output_layers_indices, (output_layers_indices.size,))
out_layers = [layer_names[i - 1] for i in output_layers_indices]

classes_to_look_for = ["person", "cell phone", "laptop", "tv", "remote"]

video_path = "/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Есть телефон/02_26_36.mp4"
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Нет телефона/00_13_48.mp4', fourcc, 20.0, (frame_width,frame_height))

phone_in_hands = False
phone_detection_timer = None
phone_hold_time = 3.0

while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        processed_frame, detected_objects = detect_objects(frame, net, out_layers, classes_to_look_for)

        if 'cell phone' in detected_objects:
            if not phone_in_hands:
                phone_detection_timer = time.time()
            phone_in_hands = True
        else:
            phone_in_hands = False

        person_in_cabin = 'person' in detected_objects

        if person_in_cabin:
            if phone_in_hands and (time.time() - phone_detection_timer) >= phone_hold_time:
                print("Телефон был в руках более 3-х секунд.")
                # Вставьте ваш код здесь, если хотите предпринять дополнительные действия после 3-х секунд
        else:
            print("Никого нет в кабине.")

        cv2.imshow('Detected Objects', processed_frame)
        out.write(processed_frame)

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в 

In [32]:
from torchvision.datasets import VOCSegmentation, VOCDetection
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import os

root = "./data"
image_set = "train"  # или "val" для тестовых данных

# Определение преобразований
transformations = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Для сегментации
seg_root = os.path.join(root, "/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Есть телефон/00_26_30.mp4")
seg_dataset = VOCSegmentation(seg_root, year="2012", image_set=image_set, transforms=transformations)

# Для обнаружения объектов
det_root = os.path.join(root, "/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Есть телефон/00_26_30.mp4")
det_dataset = VOCDetection(det_root, year="2012", image_set=image_set, transforms=transformations)


RuntimeError: Dataset not found or corrupted. You can use download=True to download it

In [ ]:
'phone', 'ipad', 'tablet', 'cup', 'book', 'no people'

In [3]:
import cv2
import numpy as np
from art import tprint

In [9]:
weights_path = '/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Нет телефона/00_09_02.mp4'
config_path = '/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Есть телефон/00_26_30.mp4'

In [10]:

net = cv2.dnn.readNet(weights_path, config_path)
layer_names = net.getLayerNames()
out_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

error: OpenCV(4.8.0) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/dnn/src/dnn_read.cpp:56: error: (-2:Unspecified error) Cannot determine an origin framework of files: /Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Нет телефона/00_09_02.mp4, /Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Есть телефон/00_26_30.mp4 in function 'readNet'


In [ ]:
def apply_yolo_object_detection(image_to_process):
    """
    Recognition and determination of the coordinates of objects on the image
    :param image_to_process: original image
    :return: image with marked objects and captions to them
    """

    height, width, _ = image_to_process.shape
    blob = cv2.dnn.blobFromImage(image_to_process, 1 / 255, (608, 608),
                                 (0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(out_layers)
    class_indexes, class_scores, boxes = ([] for i in range(3))
    objects_count = 0

    # Starting a search for objects in an image
    for out in outs:
        for obj in out:
            scores = obj[5:]
            class_index = np.argmax(scores)
            class_score = scores[class_index]
            if class_score > 0:
                center_x = int(obj[0] * width)
                center_y = int(obj[1] * height)
                obj_width = int(obj[2] * width)
                obj_height = int(obj[3] * height)
                box = [center_x - obj_width // 2, center_y - obj_height // 2,
                       obj_width, obj_height]
                boxes.append(box)
                class_indexes.append(class_index)
                class_scores.append(float(class_score))

    # Selection
    chosen_boxes = cv2.dnn.NMSBoxes(boxes, class_scores, 0.0, 0.4)
    for box_index in chosen_boxes:
        box_index = box_index
        box = boxes[box_index]
        class_index = class_indexes[box_index]

        # For debugging, we draw objects included in the desired classes
        if classes[class_index] in classes_to_look_for:
            objects_count += 1
            image_to_process = draw_object_bounding_box(image_to_process,
                                                        class_index, box)

    final_image = draw_object_count(image_to_process, objects_count)
    return final_image

In [ ]:
def draw_object_bounding_box(image_to_process, index, box):
    """
    Drawing object borders with captions
    :param image_to_process: original image
    :param index: index of object class defined with YOLO
    :param box: coordinates of the area around the object
    :return: image with marked objects
    """

    x, y, w, h = box
    start = (x, y)
    end = (x + w, y + h)
    color = (0, 255, 0)
    width = 2
    final_image = cv2.rectangle(image_to_process, start, end, color, width)

    start = (x, y - 10)
    font_size = 1
    font = cv2.FONT_HERSHEY_SIMPLEX
    width = 2
    text = classes[index]
    final_image = cv2.putText(final_image, text, start, font,
                              font_size, color, width, cv2.LINE_AA)

    return final_image

In [ ]:
def draw_object_count(image_to_process, objects_count):
    """
    Signature of the number of found objects in the image
    :param image_to_process: original image
    :param objects_count: the number of objects of the desired class
    :return: image with labeled number of found objects
    """

    start = (10, 120)
    font_size = 1.5
    font = cv2.FONT_HERSHEY_SIMPLEX
    width = 3
    text = "Objects found: " + str(objects_count)

    # Text output with a stroke
    # (so that it can be seen in different lighting conditions of the picture)
    white_color = (255, 255, 255)
    black_outline_color = (0, 0, 0)
    final_image = cv2.putText(image_to_process, text, start, font, font_size,
                              black_outline_color, width * 3, cv2.LINE_AA)
    final_image = cv2.putText(final_image, text, start, font, font_size,
                              white_color, width, cv2.LINE_AA)

    return final_image